In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split

In [2]:
# Importing training data

# train_data1 = np.load('../DL-Project/data0.npy')
# train_lab1 = np.load('../DL-Project/lab0.npy')

# train_data2 = np.load('../DL-Project/data1.npy')
# train_lab2 = np.load('../DL-Project/lab1.npy')

# train_data3 = np.load('../DL-Project/data2.npy')
# train_lab3 = np.load('../DL-Project/lab2.npy')

In [3]:
# Preprocessing - normalizing the image and adding new dimension for convolutions

# images = np.concatenate((train_data1, train_data2, train_data3)) / 255
# labels = np.concatenate((train_lab1, train_lab2, train_lab3))

# images = images[:, np.newaxis, :, :]

In [4]:
test_data = np.load('../DL-Project/data9.npy')
labels = np.load('../DL-Project/label9.npy')

images = test_data / 255
images = images[:, np.newaxis, :, :]

In [5]:
test_images = torch.tensor(images, dtype=torch.float32)
test_labels = torch.tensor(labels, dtype=torch.float32)

In [6]:
# Creating the Dataset from the tensors

class ImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

test_dataset = ImageDataset(test_images, test_labels)

# Creating DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Choosing GPU for inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
# Defining the model, loading it and moving it to the gpu

class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv_layers = nn.Sequential(
            # Convolution layer 1
            nn.Conv2d(1, 10, kernel_size=28, stride=1, padding=14),  # (N, 10, 40, 168)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (N, 10, 20, 84)
            # Convolution layer 2
            nn.Conv2d(10, 16, kernel_size=3, stride=1, padding=1),  # (N, 16, 20, 84)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (N, 16, 10, 42)
            # Convolution layer 3
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # (N, 32, 10, 42)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # (N, 32, 5, 21)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            # Fully connected layer 1
            nn.Linear(32 * 5 * 21, 256),
            nn.ReLU(),
            # Fully connected layer 1
            nn.Linear(256, 32),
            nn.ReLU(),
            # Fully connected layer 1
            nn.Linear(32, 4),
            nn.ReLU(),
            # Final layer for output
            nn.Linear(4, 1)  # Regression output
        )

    def forward(self, x):
        # Running the convolution layers
        x = self.conv_layers(x)
        # Running the fully connected layers
        x = self.fc_layers(x)
        return x

model = CNNRegressor().to(device)
model.load_state_dict(torch.load('model.pth', weights_only=True))

<All keys matched successfully>

In [9]:
# Testing the Model on the whole dataset

criterion = nn.L1Loss()

def evaluate_model(model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
    print(f"Test Loss: {test_loss/len(test_loader):.4f}")
    return [labels, outputs]

l,o = evaluate_model(model, test_loader)

Test Loss: 1.8222


In [35]:
def evaluate_acc(model, data_loader):
    model.eval()  # Set to evaluation mode
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = torch.round(model(images))
            # predicted_classes = outputs
            # print(np.shape(outputs.squeeze()), np.shape(labels))
            correct += (outputs.squeeze() == labels).sum().item()
            # print(outputs, labels)
            total += labels.size(0)
            # print(correct)

    return 100 * correct / total  # Return accuracy percentage

In [36]:
print(evaluate_acc(model, test_loader))

19.1
